# Generate EMA using daily data service - C#

### Overview
Moving averages smooth the price movements by filtering out the "noise" caused by short-term price fluctuations.They also form the building blocks for a wide range of technical indicators.

Computing moving averages over a defined number of time periods is useful to:
* identify the trend direction
* determine support and resistance levels

This sample demonstrates how to request and plot **on-demand** *exponential moving average - EMA* from a daily data service.

### Inputs/outputs
Exponential moving averages' requests require instrument's identifier, date time intervals and number of points as per inputs.

EMA reduces the lag by applying greater weight to recent data points.The weighting applied to the most recent price depends on the number of points in the moving average; The smaller the point count is, the more weight is applied to the most recent price.

N-points EMA data point is computed using the previous' *EMA* value and the wighting multiplier as follows:

$$ EMA = (Close-EMAp)*k+EMAp $$

Where:
* *Price* is the close price
* *EMAp* is the previous points' *EMA* value, *SMA* is used as the EMAp in the first calculation
* *N* is the count of points
* *$ k = \frac{2}{N+1} $* is the *Weighting Multiplier*

<div class="alert alert-block alert-info">
    <b>Note:</b> <i>EMA</i> adapts quickly to price changes than the <i>SMA</i>, when a price reverses direction, the <i>EMA</i> will reverse direction quicker than the <i>SMA</i> due to the weighting 
</div>

This sample shows how to plot a simple graph for basis technical analysis using an open source library. 

### Services used
This sample uses *gRPC requests* in order to retrieve EMA indicator from the hosted service. The queried endpoint in this script are:
* *DailyEmaSerice*: to directly retrieve daily EMA data from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.DailyAnalytics.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run daily EMA sample

### Step 1: Install packages

In [1]:
#i "nuget: file://~/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 0.*-pre*"
#r "nuget: XPlot.Plotly.Interactive"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Restore sources file://~/.nuget/packages/ https://api.nuget.org/v3/index.json Installed Packages Systemathics.Apis, 0.13.2-pre XPlot.Plotly.Interactive, 4.0.6

Loading extensions from `XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

In [2]:
using Systemathics.Apis.Helpers;
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.DailyAnalytics.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;
using XPlot.Plotly;
using XPlot.Plotly.Interactive;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [3]:
// Get token as metadata
var headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
var channel = ChannelHelpers.GetChannel();

### Step 3: Retrieve data
To request *daily ema* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* The EMA parameters

#### 3.1 Instrument selection

In [4]:
// Set instrument
var ticker = "AAPL";
var exchange = "XNGS";

#### 3.2 EMA parameters

In [5]:
// Set EMAs lengths
var longEma = 200;
var shortEma = 50;

#### 3.3 Time period selection

In [6]:
// Set data interval (we are using Google date time format)
var dateInterval = new DateInterval()
{
  StartDate = new Date { Year = 2019, Month = 01, Day = 01 },
  EndDate = new Date { Year = 2020, Month = 12, Day = 31 }
};


#### 3.4 EMAs request creation
The following code snippet creates *gRPC client*, process daily EMA requests and returns the reply.

In [7]:
// Instantiate the daily Ema service
var service = new DailyEmaService.DailyEmaServiceClient(channel);

In [8]:
// Create and process the daily long Ema request
var dailyLongEmaRequest = new DailyEmaRequest 
{ 
    Identifier = new Identifier { Exchange = exchange, Ticker = ticker }, 
    Length = longEma,
    DateInterval = dateInterval
};
var dailyLongEmaReply = service.DailyEma(dailyLongEmaRequest, headers);

// Create and process the daily short Ema request
var dailyShortEmaRequest = new DailyEmaRequest 
{ 
    Identifier = new Identifier { Exchange = exchange, Ticker = ticker }, 
    Length = shortEma,
    DateInterval = dateInterval
};
var dailyShortEmaReply = service.DailyEma(dailyShortEmaRequest, headers);

// Visualize requests' results
//display(dailyLongEmaReply.Data)
//display(dailyShortEmaReply.Data)

### Step 4: Visualize data

#### 4.1 Select data

In [9]:
// Extract the price and the EMAs
var prices = dailyLongEmaReply.Data.Select(t => new Tuple<DateTime, double>(new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Value));
var longEmaData = dailyLongEmaReply.Data.Select(t => new Tuple<DateTime, double>(new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Average));
var shortEmaData = dailyShortEmaReply.Data.Select(t => new Tuple<DateTime, double>(new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Average));

#### 4.2 Plot EMAs and daily prices
Plot EMA request results with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package. 

In [10]:
var emaGraph = new List<Scattergl>
{
    new Scattergl
    {
        name = $"{longEma}-days EMA",
        mode = "lines",
        x = longEmaData.Select(c => c.Item1),
        y = longEmaData.Select(c => c.Item2)
    },
    new Scattergl
    {
        name = $"{shortEma}-days EMA",
        mode = "lines",
        x = shortEmaData.Select(c => c.Item1),
        y = shortEmaData.Select(c => c.Item2)
    },
    new Scattergl
    {
        name = "Price",
        mode = "lines",
        x = prices.Select(c => c.Item1),
        y = prices.Select(c => c.Item2)
    }
};

var chart = Chart.Plot(emaGraph);
chart.WithTitle($"Long & short EMAs and daily prices for {ticker}");
chart.WithXTitle($"Date");
chart.Width = 1500;
chart.Height = 800;
display(chart);

<!DOCTYPE html>